## Further Data Wrangling - grouping, merging data sources, etc....

### Import Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

### Read in Data

In [2]:
df = pd.read_csv('../data/Smaller Dataset - Data Wrangling.csv', index_col = 0, parse_dates = True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7255968 entries, 2015-09-05 13:30:00 to 2020-12-27 12:13:00
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   ID            int64  
 1   IUCR          object 
 2   Primary Type  object 
 3   Description   object 
 4   Arrest        bool   
 5   Domestic      bool   
 6   Latitude      float64
 7   Longitude     float64
dtypes: bool(2), float64(2), int64(1), object(3)
memory usage: 401.4+ MB


In [4]:
type(df.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [5]:
df.index.to_series().min()

Timestamp('2001-01-01 00:00:00')

In [6]:
df.index.to_series().max()

Timestamp('2020-12-27 23:59:00')

In [7]:
# This is how many crimes resulted in arrest
df[df['Arrest']]['Arrest'].count()

1970069

In [8]:
# This is how many crimes were domestic
df[df['Domestic']]['Domestic'].count()

974228

### Group Data 

In [34]:
df.head()

,ID,IUCR,Primary Type,Description,Arrest,Domestic,Latitude,Longitude
Date,,,,,,,,
2015-09-05 13:30:00,10224738,0486,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,41.815117,-87.670000
2015-09-04 11:30:00,10224739,0870,THEFT,POCKET-PICKING,False,False,41.895080,-87.765400
2018-09-01 00:01:00,11646166,0810,THEFT,OVER $500,False,True,NaN,NaN
2015-09-05 12:45:00,10224740,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,41.937406,-87.716650
2015-09-05 13:00:00,10224741,0560,ASSAULT,SIMPLE,False,True,41.881903,-87.755121


#### First group by day and primary type

In [35]:
allcrimes_byday = df[['ID','Primary Type']].groupby(by=[df.index.date,'Primary Type']).count().rename(columns={'ID':'Count'})

In [36]:
allcrimes_byday = allcrimes_byday.reset_index().rename(columns={'level_0':'Date'}).set_index('Date')

In [37]:
allcrimes_byday.head()

,Primary Type,Count
Date,,
2001-01-01,ASSAULT,70
2001-01-01,BATTERY,296
2001-01-01,BURGLARY,66
2001-01-01,CRIM SEXUAL ASSAULT,38
2001-01-01,CRIMINAL DAMAGE,233


We have too many Primary Types so we're going to combine these into a smaller number of categories based on some research I've done on crime types.  

In [38]:
# Assign a type for a crime
def assign_type(crime):
    """categorizes a crime"""
    if crime in ['ARSON','ASSAULT','BATTERY','CRIM SEXUAL ASSAULT','CRIMINAL SEXUAL ASSAULT','DOMESTIC VIOLENCE',\
                'HOMICIDE','HUMAN TRAFFICKING','INTIMIDATION','KIDNAPPING','OFFENSE INVOLVING CHILDREN','SEX OFFENSE',\
                 'STALKING']:
        return 'Violent'
    elif crime in ['BURGLARY','CRIMINAL DAMAGE','CRIMINAL TRESPASS','MOTOR VEHICLE THEFT','ROBBERY']:
        return 'Property'
    elif crime in ['CONCEALED CARRY LICENSE VIOLATION','GAMBLING','INTERFERENCE WITH PUBLIC OFFICER','LIQUOR LAW VIOLATION',\
                  'OBSCENITY','PROSTITUTION','PUBLIC INDECENCY','PUBLIC PEACE VIOLATION','RITUALISM','WEAPONS VIOLATION']:
        return 'Public Order / Vice'
    elif crime in ['DECEPTIVE PRACTICE']:
        return 'White Collar'
    elif crime in ['NARCOTICS','OTHER NARCOTIC VIOLATION']:
        return 'Drugs'
    else:
        return 'Other'

In [39]:
allcrimes_byday['TYPE'] = allcrimes_byday.apply(lambda x: assign_type(x['Primary Type']), axis=1)
allcrimes_byday.drop(columns='Primary Type', inplace=True)

In [40]:
allcrimes_byday.head()

,Count,TYPE
Date,,
2001-01-01,70,Violent
2001-01-01,296,Violent
2001-01-01,66,Property
2001-01-01,38,Violent
2001-01-01,233,Property


In [41]:
# Now group by type
grouped_crimes_byday = allcrimes_byday.groupby(by=[allcrimes_byday.index,'TYPE']).sum()

In [42]:
grouped_crimes_byday = grouped_crimes_byday.reset_index().rename(columns={'level_0':'Date'}).set_index('Date')

In [43]:
grouped_crimes_byday.head()

,TYPE,Count
Date,,
2001-01-01,Drugs,97
2001-01-01,Other,580
2001-01-01,Property,429
2001-01-01,Public Order / Vice,48
2001-01-01,Violent,574


In [44]:
# Now pivot the table
grouped_crimes_byday = grouped_crimes_byday.pivot(columns='TYPE',values='Count')

In [45]:
grouped_crimes_byday.head()

TYPE,Drugs,Other,Property,Public Order / Vice,Violent,White Collar
Date,,,,,,
2001-01-01,97,580,429,48,574,92
2001-01-02,175,322,321,26,221,78
2001-01-03,133,322,351,32,264,49
2001-01-04,142,339,359,40,244,42
2001-01-05,185,355,367,37,270,53


#### Add total crimes column

In [46]:
grouped_crimes_byday['Total Crimes'] = grouped_crimes_byday.sum(axis=1)

In [47]:
grouped_crimes_byday['Total Crimes'].sum()

7255968

In [52]:
grouped_crimes_byday.shape

(7301, 7)

#### Group by day counting arrests and domestics

In [48]:
arrests_domestics = df[['Arrest','Domestic']].groupby(by=[df.index.date]).sum()

In [49]:
arrests_domestics[['Arrest','Domestic']] = arrests_domestics[['Arrest','Domestic']].astype('int')

In [50]:
arrests_domestics.head()

,Arrest,Domestic
2001-01-01,399,285
2001-01-02,383,146
2001-01-03,343,162
2001-01-04,377,131
2001-01-05,415,159


In [51]:
arrests_domestics.shape

(7301, 2)

#### Merge these two dataframes

In [63]:
crimes_byday = grouped_crimes_byday.join(arrests_domestics)

In [65]:
crimes_byday.head()

,Drugs,Other,Property,Public Order / Vice,Violent,White Collar,Total Crimes,Arrest,Domestic
Date,,,,,,,,,
2001-01-01,97,580,429,48,574,92,1820,399,285
2001-01-02,175,322,321,26,221,78,1143,383,146
2001-01-03,133,322,351,32,264,49,1151,343,162
2001-01-04,142,339,359,40,244,42,1166,377,131
2001-01-05,185,355,367,37,270,53,1267,415,159


#### Convert index back to datetime and set frequency

In [66]:
crimes_byday.index = pd.to_datetime(grouped_crimes_byday.index)

In [67]:
crimes_byday.index.freq = 'd'

In [68]:
crimes_byday.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7301 entries, 2001-01-01 to 2020-12-27
Freq: D
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Drugs                7301 non-null   int64
 1   Other                7301 non-null   int64
 2   Property             7301 non-null   int64
 3   Public Order / Vice  7301 non-null   int64
 4   Violent              7301 non-null   int64
 5   White Collar         7301 non-null   int64
 6   Total Crimes         7301 non-null   int64
 7   Arrest               7301 non-null   int64
 8   Domestic             7301 non-null   int64
dtypes: int64(9)
memory usage: 570.4 KB


In [70]:
# Reorder columns
crimes_byday = crimes_byday[['Total Crimes', 'Arrest', 'Domestic', 'Violent', 'Property', 'Public Order / Vice',\
                               'Drugs', 'White Collar','Other']]

In [74]:
crimes_byday.columns

Index(['Total Crimes', 'Arrest', 'Domestic', 'Violent', 'Property',
       'Public Order / Vice', 'Drugs', 'White Collar', 'Other'],
      dtype='object')

### Save Data

In [75]:
crimes_byday.to_csv('../Data/crimes_byday.csv')